## MATRIX INVERSION AS A QUBO PROBLEM

Ref  
[1.Floating-Point Calculations on a Quantum Annealer:
Division and Matrix Inversion](https://arxiv.org/pdf/1901.06526.pdf)

In this section we present an algorithm for solving a system of linear equations on a
quantum annealer. To precisely define the mathematical problem, let $M$ be a nonsingular $N × N$ real matrix, and let $Y$ be a real $N$ dimensional vector; we then wish to solve the linear equation
$$ M · x = Y$$
The linearity of the system means that there is a unique solution,
$$x = M^{−1} · Y$$

Constructing a quadratic matrix 
$$H(x) = (M x − Y)^2 = (M x − Y)^T · (M x − Y)$$
$$H(x) = x^T M^T Mx - x^T M^T Y - Y^T M x +Y^T Y$$
$$H(x) = \sum_{ijk=1}^{N} M_{ki} M_{kj} x^i x^j - 2\sum_{ij=1}^N Y_j M_{ji} x^i + ||Y^2||$$
To obtain a floating point representation of each component
of $x = (x_1, · · · , x_N)^T$ by expanding in powers of 2 multiplied by Boolean-valued variables
$$\chi^i = \sum_{r=0}^{R-1} 2^{-r}q_r^i $$
$$x^i = 2\chi^i-1$$

As before, the domains are give by $\chi^i\in [0, 2)$ and $x^i \in [−1, 3)$, and upon expressing $x$ as a function the $q__r^i$ we can recast $H(x)$ in the form  
$$H(q) = \sum_{i=1}^{N}\sum_{r=0}^{R-1} a_r^i q_r^j + \sum_{i=1}^N \sum_{i\neq j=1}^N \sum_{r=0}^{R-1} \sum_{s=0}^{R-1} b_{rs}^{ij} q_{r}^i q_s^i$$

In [2]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np
import random, math
import copy

Dimension = 2
qubits = 2
A = np.array([[3, 1], [-1, 2]])
b = np.array([-1, 5])

QM = np.zeros((2*qubits*Dimension, 2*qubits*Dimension))
for k in range(Dimension): 
    for i in range(Dimension):
        for l in range(qubits):
            cef1 = pow(2,2*l)*pow(A[k][i],2)
            cef2 = pow(2,l+1)*A[k][i]*b[k]
            po1 = 4*i + l
            po2 = 4*i + l + 2
            QM[po1][po1] = QM[po1][po1] + cef1 - cef2
            QM[po2][po2] = QM[po2][po2] + cef1 + cef2

for k in range(Dimension):
    for i in range(Dimension):
        for l1 in range(qubits-1):
            for l2 in range(l1+1,qubits):
                qcef = pow(2, l1+l2+1)*pow(A[k][i],2)
                po1 = 4*i + l1
                po2 = 4*i + l2
                QM[po1][po2] = QM[po1][po2] + qcef
                po3 = 4*i + l1 + 2
                po4 = 4*i + l2 + 2
                QM[po3][po4] = QM[po3][po4] + qcef

for k in range(Dimension):
    for i in range(Dimension-1):
            for j in range(i+1,Dimension):
                for l1 in range(qubits):
                    for l2 in range(qubits):
                        qcef = pow(2, l1+l2+1)*A[k][i]*A[k][j]
                        po1 = 4*i + l1
                        po2 = 4*j + l2
                        QM[po1][po2] = QM[po1][po2] + qcef
                        po3 = 4*i + l1 + 2
                        po4 = 4*j + l2 + 2
                        QM[po3][po4] = QM[po3][po4] + qcef
                        po5 = 4*i + l1
                        po6 = 4*j + l2 + 2
                        QM[po5][po6] = QM[po5][po6] - qcef
                        po7 = 4*i + l1 + 2
                        po8 = 4*j + l2
                        QM[po7][po8] = QM[po7][po8] - qcef

# Print Matrix Q
print("# Matrix Q is")
print(QM)

# Matrix Q is
[[ 26.  40.   0.   0.   2.   4.  -2.  -4.]
 [  0.  72.   0.   0.   4.   8.  -4.  -8.]
 [  0.   0.  -6.  40.  -2.  -4.   2.   4.]
 [  0.   0.   0.   8.  -4.  -8.   4.   8.]
 [  0.   0.   0.   0. -13.  20.   0.   0.]
 [  0.   0.   0.   0.   0. -16.   0.   0.]
 [  0.   0.   0.   0.   0.   0.  23.  20.]
 [  0.   0.   0.   0.   0.   0.   0.  56.]]


## Apply QUBO

In [6]:
from dwave.system import DWaveSampler, EmbeddingComposite
sampler_auto = EmbeddingComposite(DWaveSampler(solver={'qpu': True}))

linear = {('q1','q1'):26.0, ('q2','q2'):72.0, ('q3','q3'):-6.0, ('q4','q4'):8.0, ('q5','q5'):-13.0, ('q6','q6'):-16.0, ('q7','q7'):23.0, ('q8','q8'):56.0}

quadratic = {('q1','q2'):40.0, ('q1','q5'):2.0, ('q1','q6'):4.0, ('q1','q7'):-2.0, ('q1','q8'):-4.0, ('q2','q5'):4.0, ('q2','q6'):8.0, ('q2','q7'):-4.0, ('q2','q8'):-8.0, ('q3','q4'):40.0, ('q3','q5'):-2.0, ('q3','q6'):-4.0, ('q3','q7'):2.0, ('q3','q8'):4.0, ('q4','q5'):-4.0, ('q4','q6'):-8.0, ('q4','q7'):4.0, ('q4','q8'):8.0, ('q5','q6'):20.0, ('q7','q8'):20.0}

Q = dict(linear)
Q.update(quadratic)

sampleset = sampler_auto.sample_qubo(Q, num_reads=1000)
print(sampleset)

  q1 q2 q3 q4 q5 q6 q7 q8 energy num_oc. chain_.
0  0  0  1  0  0  1  0  0  -26.0     827     0.0
1  0  0  1  0  1  1  0  0  -21.0      81     0.0
2  0  0  1  0  1  0  0  0  -21.0      66     0.0
3  0  0  0  0  0  1  0  0  -16.0       6     0.0
4  0  0  0  1  0  1  0  0  -16.0      14     0.0
5  0  0  0  1  1  1  0  0  -13.0       2     0.0
6  0  0  0  0  1  1  0  0   -9.0       1     0.0
7  0  0  0  1  1  0  0  0   -9.0       1     0.0
8  0  0  1  0  0  0  0  0   -6.0       1     0.0
9  0  0  1  0  0  1  1  0   -1.0       1     0.0
['BINARY', 10 rows, 1000 samples, 8 variables]


In [8]:
import dimod

#print(Q)
J = dimod.qubo_to_ising(Q)
print(J)

({'q1': 23.0, 'q2': 46.0, 'q3': 7.0, 'q4': 14.0, 'q5': -1.5, 'q6': -3.0, 'q7': 16.5, 'q8': 33.0}, {('q1', 'q2'): 10.0, ('q1', 'q5'): 0.5, ('q1', 'q6'): 1.0, ('q1', 'q7'): -0.5, ('q1', 'q8'): -1.0, ('q2', 'q5'): 1.0, ('q2', 'q6'): 2.0, ('q2', 'q7'): -1.0, ('q2', 'q8'): -2.0, ('q3', 'q4'): 10.0, ('q3', 'q5'): -0.5, ('q3', 'q6'): -1.0, ('q3', 'q7'): 0.5, ('q3', 'q8'): 1.0, ('q4', 'q5'): -1.0, ('q4', 'q6'): -2.0, ('q4', 'q7'): 1.0, ('q4', 'q8'): 2.0, ('q5', 'q6'): 5.0, ('q7', 'q8'): 5.0}, 105.0)


In [10]:
sampleset = sampler_auto.sample_ising(J[0], J[1], num_reads=1000)
print(sampleset)

  q1 q2 q3 q4 q5 q6 q7 q8 energy num_oc. chain_.
0 -1 -1 +1 -1 -1 +1 -1 -1 -131.0     784     0.0
1 -1 -1 +1 -1 +1 +1 -1 -1 -126.0      55     0.0
2 -1 -1 +1 -1 +1 -1 -1 -1 -126.0     125     0.0
3 -1 -1 -1 +1 -1 +1 -1 -1 -121.0      27     0.0
4 -1 -1 -1 -1 -1 +1 -1 -1 -121.0       1     0.0
5 -1 -1 -1 -1 +1 -1 -1 -1 -118.0       2     0.0
6 -1 -1 -1 +1 +1 +1 -1 -1 -118.0       4     0.0
7 -1 -1 +1 -1 -1 -1 -1 -1 -111.0       2     0.0
['SPIN', 8 rows, 1000 samples, 8 variables]
